In [2]:
import numpy as np
import pandas as pd
import re
import string

In [3]:
def remove_functuations (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text


In [4]:
import pickle

with open('../static/model/model.pickle','rb') as f:
    model =pickle.load(f)

c:\Users\Isuru Sandaruwan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.7.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
with open('../static/model/corpora/stopwords/english','r') as file :
    sw =file.read().splitlines()

In [6]:
# Get the vocabulary(features)

vocab= pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens =vocab[0].tolist()

In [7]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()


In [8]:

def preprocessing(text):
    data=pd.DataFrame([text],columns=['tweet'])

    data["tweet"]=data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    data["tweet"]=data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data["tweet"]=data["tweet"].apply(remove_functuations)
    
    data["tweet"] = data["tweet"].str.replace(r'\d+', '', regex=True)
    
    data["tweet"]=data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    
    data["tweet"]=data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

    return data["tweet"]
    

In [9]:
def vectorizar(ds,vocabulary) :
    vectorized_list =[]

    for sentence in ds :
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] =1

        vectorized_list.append((sentence_lst))
    vectorized_lst_new =np.asarray(vectorized_list,dtype=np.float32)
    return vectorized_lst_new
    

In [10]:
def get_prediction(vectorized_text) :
    prediction =model.predict(vectorized_text)
    if prediction ==1 :
        return 'negative'
    else :
        return 'positive'

In [13]:
text= "excellent and love it"
pre_processed_text=preprocessing(text)
vectorized_text=vectorizar(pre_processed_text,tokens)
prediction=get_prediction(vectorized_text)
prediction

'positive'